# TODO find tracerRNA

example id:
8774__OBKKDPJE_00146__BritoIL_2016__M2.46.ST__bin.47;length1073;#sequences1 

In [2]:
# First created Thu Jul 11 09:51:15 CEST 2019
# Made by L-F-S
# At the University Of Trento, Italy

import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein


sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/utils/')
import filename_discrepancies

feature="Cas9" #WARNING!!! CHANGE THIS!!
#outpath="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/out/"+feature+"/"
datadir="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/"




In [3]:
# Import data
cas_dataset=pd.read_csv("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/known_"+feature+"_variants_table.csv", index_col=0)
cas_dataset.columns

Index(['Seq ID', 'Seq Description', 'Seq', 'Contig', 'Genome Name', 'Study',
       'Sample Name', 'SGB ID', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_cas1', 'prokka_cas2', 'uSGB', 'Level of estimated taxonomy',
       'Estimated taxonomy'],
      dtype='object')

In [4]:
tabellazza=pd.read_csv(datadir+"/3tabellazza/crisprcas_hits_table.csv", index_col=0)
tabellazza.columns

Index(['Unnamed: 0.1', 'Genome Name', 'Study', 'Sample Name', 'Assembler',
       'Genome Size', 'N50', '# Contigs', 'Completeness', 'Contamination',
       'Strain Heterogeneity', 'SGB ID', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_CRISPR', 'prokka_cas', 'uniref_cas'],
      dtype='object')

In [38]:
for seqid in ["OBKKDPJE_00146","LPEAHEHG_01071","CJFLGLGO_01367","FLJLKKHA_00012","LJKGNKPJ_00564","CPMHNJAN_01529", "AKIPNPCL_00899", "AJHGDLDC_00764"]:
    print("SEQ ID: ", seqid)
    print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
    print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
    print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
    print(len(cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]))
    print(cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0])
    print("----------------------------------------------------------------------")


SEQ ID:  OBKKDPJE_00146
NODE_3_length_470079_cov_10.1699
BritoIL_2016
BritoIL_2016__M2.46.ST__bin.47
1073
[('.', '179452', '179677')]
----------------------------------------------------------------------
SEQ ID:  LPEAHEHG_01071
NODE_190_length_65757_cov_25.3876
ChengpingW_2017
ChengpingW_2017__AS43raw__bin.51
1074
[('.', '9802', '10102')]
----------------------------------------------------------------------
SEQ ID:  CJFLGLGO_01367
NODE_137_length_63930_cov_6.02239
Obregon-TitoAJ_2015
Obregon-TitoAJ_2015__SM40__bin.4
1079
[('.', '44486', '44718')]
----------------------------------------------------------------------
SEQ ID:  FLJLKKHA_00012
NODE_49_length_176785_cov_12.2725
BritoIL_2016
BritoIL_2016__M1.29.ST__bin.30
979
[('.', '13625', '14320')]
----------------------------------------------------------------------
SEQ ID:  LJKGNKPJ_00564
NODE_211_length_45519_cov_7.71687
KarlssonFH_2013
KarlssonFH_2013__S420__bin.8
1002
[('.', '12991', '13885')]
-------------------------------------

In [36]:
for seqid in ["OBKKDPJE_00146","LPEAHEHG_01071","CJFLGLGO_01367","FLJLKKHA_00012","LJKGNKPJ_00564","CPMHNJAN_01529", "AKIPNPCL_00899", "AJHGDLDC_00764"]:
    spacers_list_for_casid={}
    #######questa avrà solo una key ^



    spacers_list_for_casid[seqid]=[]

    cas9_aa=cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]
    dataset=cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0]
    genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
    SGB=cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0]
    contig=cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0]
    print("\n-----------------------------------------------------\n"+feature+" id:\t"+seqid+"\nSGB:\t"+str(SGB)+"\nGenome Name:\t",\
          genomename+"\nContig:\t"+cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0],"\n-----------------------------------------------------\nCRISPR spacer sequences:")

    ###################################################################### TODO THIS MUST BE INSIDE filename_discrepancies
    # take into account filename discrepancies
    if dataset.startswith("ZeeviD"):  #TODO guarda se funziona quando finisci di girare minced_runner
             s3_dataset="ZeeviD_2015"
             #genomename=filename_discrepancies.change_to_megahit(genomename)
             s3_genomename=genomename
             try:
                 genomename=genomename.replace("ZeeviD_2015", "ZeeviD_2015_B")
                 prokkafile_ofsample=open(annodir+"/justminced/ZeeviD_2015_B/"+genomename+".crisprcas.gff.minced")
                 prokkafile_ofsample.close()
                 dataset="ZeeviD_2015_B"
             except:
                 genomename=genomename.replace("ZeeviD_2015_B", "ZeeviD_2015_A")
                 dataset="ZeeviD_2015_A"
    else:
        s3_dataset=dataset
        s3_genomename=genomename
        dataset=filename_discrepancies.s3(s3_dataset,r=True) # get working dataset name from s3 dataset name
        genomename=genomename.replace(s3_dataset, dataset)

    if filename_discrepancies.dataset_has_megahit(dataset,genomename):
        genomename=filename_discrepancies.change_to_megahit(genomename)
    ########################################################################

    # get CRISPR spacer sequence
    #spacers_list_of_contig={} initial dictionary verision with contig, but I don't need the contigname TODO this could become a function of its own
    #temporarily commented out 

    mincedCRISPRfilename=datadir+"1crisprsearch/out/"+s3_dataset+"/"+s3_genomename+".fa.minced.out"
    f=open(mincedCRISPRfilename, "r")
    for line in f.readlines():
        if line[0]== "1" or line[0]== "2" or line[0]== "3" or line[0]== "4" or line[0]== "5" or \
        line[0]== "6" or line[0]== "7" or line[0]== "8" or line[0]== "9" or line[0]== "0":
            if not line.split("\t")[3]=="\n":
                spacers_list_for_casid[seqid].append(line.split("\t")[3])
    f.close()

    #print(spacers_list_for_casid[seqid],"\n")



    # get whole locus sequence todo this could become a function of its own, dentro utils, gli dai genomename della cas e/o seqid della cas (anzi famo solo seqid, che magari ci sn doppioni in genomi) e ti dà tt il locus

        # print start and stop of CRISPR
    unostranosplit=cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0].split("\', \'")
    crispr_start, crispr_stop = int(unostranosplit[1]), int(unostranosplit[2].strip("\')]\""))

        # > get start and stop position of all Cass. questo devi metterlo nella tabella di cas non ci sono santi.

    prokka_anno=tabellazza[tabellazza["Genome Name"]==s3_genomename]["prokka_cas"].iloc[0]
    # so già che all casses inside here are on the same contig which is 'contig', xke le ho estratte per essere così

    #passaggi presi da 2fill_table:
    prokkaslist=prokka_anno.split(">")
    prokkaslist.pop(0)
    # get contigname of current effectorcas   
    cas_position={}
    for casanno in prokkaslist:
        unostranosplit=casanno.split("Name=")
        casnumber=unostranosplit[1][3] #because we know that we are loooking at 'good' loci, this is just going to be 9, 1 or 2
        unaltrostranosplit=casanno.split("\t")
        cas_start=unaltrostranosplit[3]
        cas_stop=unaltrostranosplit[4]
        cas_position["Cas"+casnumber]=[int(cas_start),int(cas_stop)]

    #

    genomefilename= "/shares/CIBIO-Storage/CM/scratch/tmp_projects/epasolli_darkmatter/allcontigs/ALLreconstructedgenomes/"+str(SGB)+"/"+genomename+".fa"
    print("Retrieving locus")
    for record in SeqIO.parse(genomefilename,"fasta"):
        if record.id.startswith(contig):
            #test se effettivamente la seq di cas è dove voglio io

            print("CRISPR si trova", crispr_start, crispr_stop)
            print("Cas9 si trova", cas_position["Cas9"])
            print("Cas2 si trova", cas_position["Cas2"])
            print("Cas1 si trova", cas_position["Cas1"])

            #estrai la posizione di cas9 dal contig, e traducila
            cas9start=[int(x) for x in cas_position["Cas9"]][0]
            cas9stop=[int(x) for x in cas_position["Cas9"]][1]
            cas9_nnseq=record.seq[cas9start-1:cas9stop-3] #gff should have a 1-based positional annotation (ma sto andando a occhio finchè non sono uguali, è già un oggetto Bio.Seq

            my_translated_cas9=cas9_nnseq.transcribe().translate()

            print(my_translated_cas9)
            print("++++++++++annotatedcas9+++++++++++++\n",cas9_aa)

            print(len(cas9_nnseq)/3,len(cas9_aa))


            print("Se la sequenza di Cas9 annotata da prokka e il mio indexing sono corretti, questo è", my_translated_cas9==cas9_aa)

            #extact a locus, 200 bp a monte e a valle dei cosi che ho:
            starts, stops=[startstop[0]for startstop in cas_position.values()], [startstop[1]for startstop in cas_position.values() ]
            minimo=min([crispr_start]+starts)
            print("start: ", minimo)
            massimo=max([crispr_stop]+stops)
            print("end: ", massimo)

            print("let's say the locus is +-200 bp a monte e a valle di start e end. BELLAPETTUTT")


    # certo che 1) avrei potuto mettere nu indicazione di poosizione dentro il cas9, ma vbb 2) trovare il modo di capire le posizioni di cirspr dentro l'arrai a cosa si riferiscono. e qst lo posso vede ora.





-----------------------------------------------------
Cas9 id:	OBKKDPJE_00146
SGB:	8774
Genome Name:	 BritoIL_2016__M2.46.ST__bin.47
Contig:	NODE_3_length_470079_cov_10.1699 
-----------------------------------------------------
CRISPR spacer sequences:
Retrieving locus
CRISPR si trova 179452 179677
Cas9 si trova [174787, 178008]
Cas2 si trova [178941, 179336]
Cas1 si trova [178008, 178898]
MYTKEIIWGFDIGKGSLGEAVRVGNEFKHVQSLEIAPDFAETKTTAAARRAWRTREAHKARELWLEKCLSEAGIEVLLRRKVGVVDGAWKLVSKGDVRLEREFPPAGEDLCYNSIALRCKLLLGEKLESWQVFKALNSAIQNRGYDANIPWGDEDAADGGDGSSRKSSDSDASGDGESYAEKLAHYEREFDGILAELDNREKYEFPCFFKAAKMGLWSPENPDFVGVRIDNTAEKAKGYVMPREAVVREFRALVNAAALQYPNLKGKADFIMFGRACEPYASYNPELRKRFGLKRGAATDWTALGQKTPRFDNRVIDKCKLIPRLNVCKIKPLDECKTDDDFLSYTITLSLKLLNLRFFRGAGVDSLSFDEFLRAFELAKNSKFKLGKAALKKFFKTIGAEVMDESQSEIEQPRNGGRAAFSRPAMRILRDLIFSGKSPAEFYATAIENIANTDKNKGLVAGDLDFLKLMGDAPWSGIFIPDVGLSDFADASAAAREAQINKLIGSQNDPIVRHRLSFFYERIKSLEAKFGVPDKIVLEFVRDDFLGKKAKSDLNKAIKTRFDEKKKLAEDLDSKREEFGFKYASKKMLLKFELYKKQGGECVYT


-----------------------------------------------------
Cas9 id:	LJKGNKPJ_00564
SGB:	14454
Genome Name:	 KarlssonFH_2013__S420__bin.8
Contig:	NODE_211_length_45519_cov_7.71687 
-----------------------------------------------------
CRISPR spacer sequences:
Retrieving locus
CRISPR si trova 12991 13885
Cas9 si trova [9720, 12728]
Cas2 si trova [15225, 15542]
Cas1 si trova [14325, 15215]
MEITINREIGKLGLPRHLVLGMDPGIASCGFALIDTANHEILDLGVRLFDSPTHPKTGQSLAVIRRGFRSTRRNIDRTQARLKHCLQILKAYGLIPQDATKEYFHTTKGDKQPLKLRVDGLDRLLNDREWALVLYSLCKRRGYIPHGEGNQDKSSEGGKVLSALAANKEAIAETSCRTVGEWLAQQPQSRNRGGNYDKCVTHAQLIEETHILFDAQRSFGSKYASPEFETAYIEVCDWERSRKDFDRRTYDLVGHCSYFPTEKRAARCTLTSELVSAYGALGNITIIHENGTSRALSATERDECIAILFSCEPIRGNKDCAVKFGALRKALDLSSGDYFKGVPAADEKTREVYKPKGWRVLRNTLNATNPILLQRLRDDRNLADAVMEAVAYSSALPVLQEQLQGLPLSEAEIEALCRLPYSSKALNGYGNRSKKALDMLLDCLEEPEVLNLTQAENDCGLLGLRIAGAQLERSDRLMPYETWIERTGRTNNNPVVIRSMSQMRKVVNAICRKWGVPNEIHVELDRELRLPQRAKDEIAKANKKNEKNRERIAGQIAELRGCAADEVTGKQIEKYRLWEEQEYFDLYTGAKIEVDRLISDDTYTQIDHILPFSR

In [ ]:

s3_dataset=row.Study
s3_genomename=row["Genome Name"]
contig=row.Contig
idd=row["Seq ID"]
print("\n-----------------------------------------------------\nidd:",idd,"\tgenome",s3_genomename,"\tlen:",len(row.Seq),"\n-----------------------------------------------------\n")


# 1 get CRISPR spacer sequence
repeats_list_for_contig=[]
pilercrCRISPRfilename=CRISPRdir+"/"+s3_dataset+"/"+s3_genomename+".fa.pilercr.out"
#print(mincedCRISPRfilename)
we_are_in_the_right_contig=False
laprossimalinebuona=False
sononelsummary=False
f=open(pilercrCRISPRfilename, "r")
n=0
for line in f.readlines():
    if line.startswith("SUMMARY BY POSITION"):
        sononelsummary=True

    if sononelsummary:
        if laprossimalinebuona:
        #    print("BA")
         #   print(line.split()[-1])
            repeats_list_for_contig.append(line.split()[-1])  #2 for repeat, 3 for spacer
            we_are_in_the_right_contig=False
            laprossimalinebuona=False
            sononelsummary=False


        if line.startswith(">"):
#                print(line)
            if contig==line.lstrip(">").strip():
#                  print("contig giust")
                we_are_in_the_right_contig=True
        if we_are_in_the_right_contig:
            if line.startswith("="):
#                  print("line giust")
                laprossimalinebuona=True

f.close()
print(repeats_list_for_contig)  #usually just one!
    
    # ho la lista di repeats:
    #fai tracrRNA
